<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/Keras_2_hidden_layer_Keras_DeepFFNN_Churn_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
import pandas as pd

In [0]:
import numpy as np

import matplotlib.pyplot as plt


In [0]:
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.shape

(7043, 21)

In [6]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [7]:
df['customerID'].nunique()

7043

In [0]:
df = df.drop(columns="customerID")

In [9]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [10]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [0]:
#Many variables here are categorical and need to be encoded

In [12]:
df['MultipleLines'].nunique()

3

In [13]:
df['InternetService'].nunique()

3

In [14]:
df['Contract'].nunique()

3

In [15]:
df['Churn'].nunique()

2

In [0]:
#convert target column to binary 1 and 0 values

In [0]:
df['Churn'] = df['Churn'].replace(regex='Yes', value=1)
df['Churn'] = df['Churn'].replace(regex='No', value=0)

In [18]:
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


In [19]:
#Split into train and validation datasets

from sklearn.model_selection import train_test_split
train, val = train_test_split(df, train_size=0.80, test_size=0.20, stratify=df['Churn'], random_state=42)
train.shape, val.shape

((5634, 20), (1409, 20))

In [20]:
train['Churn'].value_counts(normalize=True)

0    0.734647
1    0.265353
Name: Churn, dtype: float64

In [21]:
val['Churn'].value_counts(normalize=True)

0    0.734564
1    0.265436
Name: Churn, dtype: float64

In [0]:
#Thus, we have equal distribution of Churn in both train and val. and both datasets are balanced.

####Baseline target (churn): Zero features in train dataset

In [23]:
# Train dataset baseline
target = 'Churn'
y_train = train[target]
y_train.value_counts(normalize=True)

0    0.734647
1    0.265353
Name: Churn, dtype: float64

In [0]:
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)


In [25]:
#baseline accuracy if we guessed the majority class for every prediction on train dataset

from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

#Baseline accuracy with no features = 73.46%

0.7346467873624423

###Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
log_reg = LogisticRegression()

In [0]:
# 3. Arrange X feature matrices (already did y target vectors)
target = 'Churn'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]

In [0]:
#Apply encoding on train, validation and test datasets
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
train_encoded = encoder.fit_transform(X_train)
val_encoded = encoder.fit_transform(X_val)

In [0]:
# 4. Scale the data before doing logistic regression
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_encoded)
X_val_scaled = scaler.transform(val_encoded)

In [31]:
# 4. Fit the model
log_reg.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
# 5. Apply the model to new data.
# The predictions look like this ...
print('Logistic Regression Validation Accuracy', log_reg.score(X_val_scaled, y_val))

Logistic Regression Validation Accuracy 0.7714691270404542


####Using Keras Dense Feed forward NN for classification

In [0]:
import tensorflow

from tensorflow import keras

In [0]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, PReLU

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np

In [0]:

model = Sequential()

model.add(Dense(128, input_shape=(X_train_scaled.shape[1:])))
model.add(tensorflow.keras.layers.PReLU())
model.add(Dropout(0.2)) 

model.add(Dense(64))  #Hidden Layer 1
model.add(tensorflow.keras.layers.PReLU())
model.add(Dropout(0.2)) 

model.add(Dense(32)) #Hidden Layer 2
model.add(tensorflow.keras.layers.PReLU())
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid')) #use 1 neuron for binary classification in output layer sigmoid for binary classification

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy']) 

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               2560      
_________________________________________________________________
p_re_lu (PReLU)              (None, 128)               128       
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 64)                64        
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2

In [38]:
#Fit the model

stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1)


history = model.fit(X_train_scaled, y_train, epochs=20, 
          validation_data=(X_val_scaled, y_val),
          callbacks=[stop], batch_size=10)

Epoch 1/20
564/564 [==============================] - 1s 2ms/step - loss: 0.4572 - binary_accuracy: 0.7833 - val_loss: 0.4329 - val_binary_accuracy: 0.7949
Epoch 2/20
564/564 [==============================] - 1s 2ms/step - loss: 0.4322 - binary_accuracy: 0.7930 - val_loss: 0.4515 - val_binary_accuracy: 0.7892
Epoch 00002: early stopping


###Validation accuracy using 2 hidden layer Deep FFNN: 79.49%
